In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt
from statsmodels.tsa.exponential_smoothing.ets import ETSModel


ModuleNotFoundError: No module named 'statsmodels'

In [ ]:

#using single family home prices historical dataset (zillow has a few other options as well to try including w/ condos, Xbedrooms, etc.)
sfh_prices = pd.read_csv("/Users/afrazier/Documents/personal/real_estate/data/single_family_historical.csv")
print(sfh_prices.index.size)


In [ ]:
#pre-processing, filter out zipcodes without significant populations
twenty_ten_census_zip_populations = pd.read_csv("data/population_by_zip_2010.csv")
zip_populations = twenty_ten_census_zip_populations.groupby('zipcode')['population'].sum().reset_index()
prices_with_pops = sfh_prices.merge(zip_populations, left_on="RegionName", right_on="zipcode")
print(prices_with_pops.index.size)
#filter to zips with atleast 25K people living there
min_population = 25000
prices_with_pops = prices_with_pops[prices_with_pops.population>min_population]
print(prices_with_pops.index.size)
sfh_prices = prices_with_pops

In [ ]:
#melt the date,price into multiple rows instead of one wide table
date_cols = [x for x in sfh_prices.columns if sfh_prices[x].dtype=='float64']
melted_prices = pd.melt(sfh_prices, id_vars=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName','State', 'City', 'Metro', 'CountyName'], value_vars=date_cols)
melted_prices = melted_prices.rename(columns = {"variable": "date", "value": "price"})
melted_prices.date = pd.to_datetime(melted_prices.date)
# remove years/zips with null prices -> 693977/5044869 (~14%) null prices 
melted_prices = melted_prices[~(melted_prices.price.isnull())]
#calculate monthly changes in price
melted_prices['monthly_price_change'] = melted_prices.sort_values(["RegionName", "date"])['price'].diff()


In [ ]:
#add in the year distnace from the last date so we can perform regression on the price
max_date = melted_prices.date.max()
melted_prices['months_difference'] = (melted_prices.date-max_date)/(np.timedelta64(1, 'M'))

#filter to only datapoints from the last 5 years (12*5=16)
#can try other values here as it might be nice to include the 2008 crash as well
melted_prices = melted_prices[melted_prices.months_difference>-60]



In [ ]:
#calculate slope, std, and number datapoints
zipcode_slopes = melted_prices.sort_values(["RegionName", "date"]).groupby("RegionName").apply(
            lambda x: linregress(x.months_difference, x['price'])[0]).round()
zipcode_slopes.name = "monthly_price_slope"
zipcode_price_change_stds = melted_prices.groupby("RegionName")['monthly_price_change'].std().round()
zipcode_price_change_stds.name = "monthly_price_change_std"
zipcode_datapoints = melted_prices.groupby("RegionName")['price'].size()
zipcode_datapoints.name = "total_data_points"

In [ ]:
ten_year_slope_and_stds = pd.concat([zipcode_slopes, zipcode_price_change_stds, zipcode_datapoints], axis=1)
#lets filter to zipcodes with atleast 3 years of data?
ten_year_slope_and_stds =  ten_year_slope_and_stds[ten_year_slope_and_stds.total_data_points>=36]

ten_year_slope_and_stds.plot.scatter(x="monthly_price_slope", y="monthly_price_change_std")

In [ ]:
ten_year_slope_and_stds['sharpe_ratios'] =  ten_year_slope_and_stds['monthly_price_slope']/ten_year_slope_and_stds['monthly_price_change_std']


#Lets Just look at zip codes with >2k a month
growth = ten_year_slope_and_stds[ten_year_slope_and_stds.monthly_price_slope>10000]
growth.sort_values('sharpe_ratios', ascending=False)[:25]

In [ ]:
#interesting zipcodes 
# with slower appreciation but low std (risk)
#1. 84532 - Moab, UT (arches?!)
#2. 84032 - Wasatch County, Utah
#3. 98632 - Longview, WA 98632


#with larger appreciaion
#and still relatively low risk
#1. 91108 - Pasadena, CA  -> no houses in this specific zip?
#2. 90211 - Beverly Hills, CA 90211

melted_prices[melted_prices.RegionName==91108][['date', 'price']].set_index("date").plot()


In [ ]:
#why no austin, nash in these lists? which filter is removing them?


#east side has 4K appreciation with lowish std. -> 0.57 sharpe

# 78702, east side austin
ten_year_slope_and_stds[ten_year_slope_and_stds.index==78702]

fig, ax = plt.subplots()


price_plot = melted_prices[melted_prices.RegionName==78702][['date', 'price']].set_index("date").plot(ax=ax, label="actual")
linear_fit_x = price_plot.lines[0].get_xdata()
linear_fit_y = np.arange(400000,400000+4030.0*len(linear_fit_x), 4030)
# linear_fit_x = [x.to_timestamp() for x in linear_fit_x]
ax.plot(linear_fit_x, linear_fit_y, label="linear_fit")
plt.legend()
plt.show()

#austin has much higher appreciation in the last few years. can we build a slightly better forecast than linear?


In [ ]:
#requirements -> good sharpe ratio of risk to return. ideally >0.5

# forecasted annual property value increase of >20K
#have a basic linear model but might want to look into other things here


#things to add
# population is growing at decent rate
# major school, grocery store, and hospital within 10-20 miles of home
# Average salary/income shows a positive trend for this place too! 


# rental yield is strong. ie, i can make 1% of house cost in rent each year? (check vs BRRRRR book rule)
# or I can make X dollars in rent each year. 